In [7]:
import tensorflow as tf
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from itertools import combinations

mnist = tf.keras.datasets.mnist

In [9]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
n_train = x_train.shape[0]
n_test = x_test.shape[0]

x_train, x_test = x_train/127.5 - 1, x_test/127.5 - 1

nb_features = np.prod(x_train.shape[1:])

x_train.resize((n_train, nb_features))
x_test.resize((n_test, nb_features))

### Task 1. Data visualisation (10 points)

In [78]:
pca = PCA(n_components=2)
x_train_pca = pca.fit_transform(x_train)

In [ ]:
plt.figure(figsize=(8, 5))
for digit in range(10):  # Loop over 10 classes
    plt.scatter(x_train_pca[y_train == digit, 0], x_train_pca[y_train == digit, 1], label=f'Class {digit}', alpha=0.7)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # Ensure unique labels
plt.legend(by_label.values(), by_label.keys())

plt.title('PCA of Dataset with 10 Classes')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()


In [ ]:
class_pairs = list(combinations(range(10), 2))

# Create subplots for all class pairs
fig, axes = plt.subplots(len(class_pairs) // 7 + (len(class_pairs) % 7 != 0), 7, figsize=(8, 8))

axes = axes.flatten()

for idx, (class1, class2) in enumerate(class_pairs):
    ax = axes[idx]
    
    # Plot data points for the two classes
    ax.scatter(x_train_pca[y_train == class1, 0], x_train_pca[y_train == class1, 1], alpha=0.7, s=10, label=f'Class {class1}')
    ax.scatter(x_train_pca[y_train == class2, 0], x_train_pca[y_train == class2, 1], alpha=0.7, s=10, label=f'Class {class2}')
    
    ax.set_title(f'{class1} vs {class2}', fontsize=6)
    ax.set_aspect('equal')  # Make each plot square-shaped
    ax.set_xticks([])  # Remove x-axis ticks
    ax.set_yticks([])  # Remove y-axis ticks
    ax.set_xlabel('')
    ax.set_ylabel('')

# Remove any empty subplots if number of plots is not a perfect multiple of columns
for idx in range(len(class_pairs), len(axes)):
    fig.delaxes(axes[idx])

# Adjust layout for minimal white space
plt.subplots_adjust(wspace= 0.02, hspace=0.33)
plt.show()

* Why is PCA a good option to visualise data?
* Add plots to your report and discuss your observations.
* Which classes can be linearly separated?


In [81]:
def digit_pairs(digit1, digit2):
    cond = (y_train==digit1) + (y_train==digit2)
    binary_x_train = x_train[cond, :]
    binary_y_train = y_train[cond].astype(int)
    binary_y_train[binary_y_train == digit1] = -1
    binary_y_train[binary_y_train == digit2] = 1
    return binary_x_train, binary_y_train

In [82]:
binary_x_train, binary_y_train = digit_pairs(0, 1)

In [ ]:
binary_X_pca = pca.fit_transform(binary_x_train)

plt.figure(figsize=(5, 3.5))
for digit in [-1, 1]:  # Loop over 10 classes
    plt.scatter(binary_X_pca[binary_y_train == digit, 0], binary_X_pca[binary_y_train == digit, 1], label=f'Class {digit}', alpha=0.7)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # Ensure unique labels
plt.legend(by_label.values(), by_label.keys())

plt.title('PCA of Dataset with 10 Classes')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

### Task 2. Perceptrons (10 points)

In [ ]:
def predict(x, w, b):
    weighted_sum = np.dot(x, w)
    result = 1 if weighted_sum > b else -1
    return result

def optimize(x, y):
    iteration = 0
    error = np.inf
    m, n = x.shape
    w = np.random.uniform(-1, 1, n)
    b = np.random.uniform(-1, 1)
    learning_rate = 0.002
    while (iteration <= 1000) & (error > 1e-3):
        error = 0
        fp, fn = 0, 0
        predictions  = []
        for sample, target in zip(x, y):
            prediction = predict(sample, w, b)
            predictions.append(prediction)
            if prediction != target:
                if prediction > target:
                    fp += 1
                else:
                    fn += 1
                error = error + 1
                w = w + learning_rate*(target-prediction)*sample
                b = b + learning_rate*(target-prediction)

        iteration += 1        
        print("Iteration:", iteration, 'with error:', error, fp, fn)
    return predictions

binary_x_train, binary_y_train = digit_pairs(1, 7)
preds = optimize(binary_x_train, binary_y_train)

In [ ]:
for i in zip(preds, binary_y_train):
    if i[0] != i[1]:
        print(i)
print(len(preds))

In [86]:
digit = binary_x_train[8].reshape(28, 28)

In [ ]:
binary_y_train[:6]

In [88]:
import numpy as np
import matplotlib.pyplot as plt

def print_digit(sample_no):
    digit = sample_no.reshape(28, 28)

    plt.imshow(digit, cmap='gray')  # Use 'gray' colormap for grayscale images
    plt.axis('off')  # Hide the axes
    plt.show()

In [89]:
sorted_labels = np.argsort(y_train)
x_train_sorted = x_train[sorted_labels]

In [ ]:
print_digit(x_train_sorted[59999])

## 3

In [15]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


import datetime

In [29]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train/255
y_train = np.eye(10)[y_train]
X_test = X_test/ 255
y_test = np.eye(10)[y_test]

n_train = X_train.shape[0]
n_test = X_test.shape[0]

X_train.resize((n_train, nb_features))
X_test.resize((n_test, nb_features))

model = Sequential()

model.add(Dense(1000, input_shape=(X_test.shape[1],), activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,          # Log weight histograms every epoch
    write_graph=True,          # Log the computation graph
    write_images=True,         # Log model weights as images
    update_freq='epoch'        # How often to write logs (defaults to every batch)
)

history = model.fit(X_train, y_train, 
                    epochs=5,               # Number of epochs
                    batch_size=128,           # Batch size
                    validation_split=0.2,
                    callbacks=[tensorboard_callback])     # Split some of the data for validation

test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/5
375/375 [==============================] - 8s 21ms/step - loss: 0.2148 - accuracy: 0.9362 - val_loss: 0.1168 - val_accuracy: 0.9650
Epoch 2/5
375/375 [==============================] - 8s 21ms/step - loss: 0.0789 - accuracy: 0.9761 - val_loss: 0.1023 - val_accuracy: 0.9688
Epoch 3/5
375/375 [==============================] - 7s 18ms/step - loss: 0.0489 - accuracy: 0.9841 - val_loss: 0.0978 - val_accuracy: 0.9704
Epoch 4/5
375/375 [==============================] - 8s 20ms/step - loss: 0.0348 - accuracy: 0.9891 - val_loss: 0.0821 - val_accuracy: 0.9764
Epoch 5/5
313/313 [==============================] - 3s 8ms/step - loss: 0.0977 - accuracy: 0.9716


In [13]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir logs/fit

In [57]:
model2 = Sequential()

model2.add(Dense(1000, input_shape=(X_test.shape[1],), activation='relu'))
model2.add(Dense(3000, activation='relu'))
model2.add(Dense(1000, activation='relu'))
model2.add(Dense(500, activation='relu'))
model2.add(Dense(10, activation='softmax'))

model2.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,          # Log weight histograms every epoch
    write_graph=True,          # Log the computation graph
    write_images=True,         # Log model weights as images
    update_freq='epoch'        # How often to write logs (defaults to every batch)
)

history = model2.fit(X_train, y_train, 
                    epochs=5,               # Number of epochs
                    batch_size=128,           # Batch size
                    validation_split=0.2,
                    callbacks=[tensorboard_callback])     # Split some of the data for validation

test_loss, test_acc = model2.evaluate(X_test, y_test)

Epoch 1/5
375/375 [==============================] - 23s 58ms/step - loss: 0.2277 - accuracy: 0.9293 - val_loss: 0.1441 - val_accuracy: 0.9574
Epoch 2/5
375/375 [==============================] - 22s 58ms/step - loss: 0.0988 - accuracy: 0.9711 - val_loss: 0.1234 - val_accuracy: 0.9653
Epoch 3/5
375/375 [==============================] - 20s 55ms/step - loss: 0.0670 - accuracy: 0.9803 - val_loss: 0.1030 - val_accuracy: 0.9712
Epoch 4/5
375/375 [==============================] - 20s 53ms/step - loss: 0.0500 - accuracy: 0.9849 - val_loss: 0.1010 - val_accuracy: 0.9707
Epoch 5/5
313/313 [==============================] - 4s 13ms/step - loss: 0.1014 - accuracy: 0.9741
